# TESCAN Support

In [ ]:
%load_ext autoreload
%autoreload 2

import os
from pprint import pprint

import matplotlib.pyplot as plt

from fibsem import acquire, utils
from fibsem.config import CONFIG_PATH
from fibsem.microscopes.tescan import TescanMicroscope
from fibsem.structures import BeamType

TESCAN_CONFIGURATION = os.path.join(CONFIG_PATH, "tescan-configuration.yaml")

microscope: TescanMicroscope
microscope, settings = utils.setup_session(config_path=TESCAN_CONFIGURATION, 
                                           manufacturer="Tescan", ip_address="localhost")

In [ ]:
settings.image.hfw = 900e-6
settings.image.dwell_time = 0.2e-6
settings.image.autocontrast = False

images = acquire.take_reference_images(microscope, settings.image)

fig, ax = plt.subplots(1, 2, figsize=(10, 10))
ax[0].imshow(images[0].data, cmap='gray')
ax[1].imshow(images[1].data, cmap='gray')
plt.show()

pprint(images[1].metadata.to_dict())

In [ ]:
state = microscope.get_microscope_state()

pprint(state.to_dict())

In [ ]:
sem_settings = microscope.get_beam_settings(BeamType.ELECTRON)
pprint(sem_settings.to_dict())

fib_settings = microscope.get_beam_settings(BeamType.ION)
pprint(fib_settings.to_dict())


In [ ]:
# because some params aren't available via the api, we need to store them internally and update them when an image is taken
# dwell_time, beam_shift, stigmation, preset

In [ ]:
microscope.set_microscope_state(state)

In [ ]:
print(microscope.get("presets", BeamType.ION))

In [ ]:
settings.image.beam_type = BeamType.ION
settings.image.hfw = 1000e-6
image = microscope.acquire_image(image_settings=settings.image)

plt.imshow(image.data, cmap='gray')
plt.show()

In [ ]:
from fibsem.structures import FibsemStagePosition
import math
microscope.move_stage_absolute(FibsemStagePosition(r=math.radians(180), z=0.006))
microscope.move_stage_absolute(FibsemStagePosition(t=math.radians(45)))

print(microscope.get_stage_position())

In [ ]:
microscope.move_flat_to_beam(beam_type=BeamType.ELECTRON)

In [ ]:
from fibsem.structures import FibsemRectangle

settings.image.beam_type = BeamType.ELECTRON
settings.image.hfw = 1000e-6
settings.image.reduced_area = None
image = microscope.acquire_image(image_settings=settings.image)

print(image.data.shape)
# pprint(image.metadata.to_dict())

plt.imshow(image.data, cmap='gray')
plt.plot(384, 256, "y+", ms=50)
plt.plot(1151, 767, "y+", ms=50)  #
# 1151 767

plt.show()
settings.image.reduced_area = FibsemRectangle(0.25, 0.25, 0.5, 0.5)

image = microscope.acquire_image(image_settings=settings.image)

print(image.data.shape)
# pprint(image.metadata.to_dict())

plt.imshow(image.data, cmap='gray')
plt.show()

In [ ]:
pos = microscope.get_stage_position()

In [ ]:
stage_position = microscope.get_stage_position()
print(stage_position.to_dict())
from fibsem.structures import FibsemStagePosition
import math

microscope.move_stage_absolute(FibsemStagePosition(x=0, y=0, z=0.005, t=0, r=0))

In [ ]:
from fibsem.microscopes.tescan import to_tescan_stage_position
stage_position = microscope.get_stage_position()
print(to_tescan_stage_position(stage_position))

In [ ]:
initial_position = microscope.get_stage_position()

In [ ]:
initial_position.r = math.radians(180)
microscope.move_stage_absolute(initial_position)

In [ ]:
microscope.move_flat_to_beam(beam_type=BeamType.ION)

In [ ]:
print(microscope.system.stage)

In [ ]:
microscope.vertical_move(dy=0.01)

### MILLING


In [ ]:
%load_ext autoreload
%autoreload 2

import os
from pprint import pprint

import matplotlib.pyplot as plt

from fibsem import acquire, utils
from fibsem.config import CONFIG_PATH
from fibsem.microscopes.tescan import TescanMicroscope
from fibsem.structures import BeamType

TESCAN_CONFIGURATION = os.path.join(CONFIG_PATH, "tescan-configuration.yaml")

microscope: TescanMicroscope
microscope, settings = utils.setup_session(config_path=TESCAN_CONFIGURATION, 
                                           manufacturer="Tescan", ip_address="localhost")


from pprint import pprint
protocol = utils.load_protocol(r"C:\Users\cleev\Documents\github\openfibsem\autolamella\autolamella\protocol\protocol-on-grid.yaml")

from fibsem.milling import get_milling_stages


stages = get_milling_stages("mill_rough", protocol["milling"])

from fibsem.milling import mill_stages, setup_milling, draw_patterns

stages[0].milling.preset = "30 keV; 20 nA"
stages[0].pattern.depth = 0.2e-6


print(microscope.get("presets", BeamType.ION))


In [ ]:
mill_stages(microscope, [stages[0]])